In [2]:
import requests
r = requests.get('https://www.eia.gov/state/')

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="EIA" name="agency"/>
  <meta content="general" name="audience"/>
  <meta content="Energy Data and Statistics, Information, and Maps" name="subject"/>
  <meta content="Petroleum prices, supply and demand information from the Energy Information Administration - EIA - Official Energy Statistics from the U.S. Government" name="Description"/>
  <meta content="crude oil, spot prices, retail gasoline prices, diesel fuel prices, weekly supply estimates,
crude oil inventories, stocks" name="Keywords"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content="en-us" http-equiv="Content-Language"/>
  <meta content="all" name="robots"/>
  <meta content="EIA - Energy Information Administration" name="

In [4]:
for i in soup.find_all('a'):
    print(i.get('href'))

None
https://twitter.com/EIAgov
None
https://www.facebook.com/eiagov
None
#page-sub-nav
/
#
#menu
#menu
#menu
#menu
#menu
#menu
#menu
#nav-sources
#nav-topics
#nav-geography
#nav-tools
#nav-learn
#nav-news
/petroleum/
/petroleum/weekly/
/petroleum/supply/weekly/
/naturalgas/
http://ir.eia.gov/ngs/ngs.html
/naturalgas/weekly/
/electricity/
/electricity/monthly/
/electricity/data/browser/
/consumption/
/consumption/residential/
/consumption/commercial/
/coal/
/coal/production/quarterly/
/beta/coal/data/browser/
/renewable/
/renewable/afv/
/nuclear/
/nuclear/outages/
/totalenergy/
/totalenergy/data/monthly/
/totalenergy/data/annual/
/analysis/
/outlooks/steo/
/outlooks/aeo/
/outlooks/ieo/
/finance/
/finance/data.php
/finance/markets/crudeoil/
/environment/
/environment/emissions/carbon/
/environment/emissions/state/analysis/
/environment/emissions/state/
/special/disruptions/
/special/disruptions/
/state/
/state/seds/
/maps/
/state/maps.php
/special/gulf_of_mexico/
/beta/international/
/b

In [5]:
#?sid=AK#SupplyDistribution
base_url = 'https://www.eia.gov/state/data.php'
state_links = []
for i in soup.find_all('a'):
    url = i.get('href')
    if url and '?sid=' in url and '?sid=US' not in url and '?sid=DC' not in url:
        state_links.append(url)
        
state_links = state_links[0:50]  #rest of the links are not states
print(len(state_links))

50


In [6]:
state_links

['?sid=AL',
 '?sid=AK',
 '?sid=AZ',
 '?sid=AR',
 '?sid=CA',
 '?sid=CO',
 '?sid=CT',
 '?sid=DE',
 '?sid=FL',
 '?sid=GA',
 '?sid=HI',
 '?sid=ID',
 '?sid=IL',
 '?sid=IN',
 '?sid=IA',
 '?sid=KS',
 '?sid=KY',
 '?sid=LA',
 '?sid=ME',
 '?sid=MD',
 '?sid=MA',
 '?sid=MI',
 '?sid=MN',
 '?sid=MS',
 '?sid=MO',
 '?sid=MT',
 '?sid=NE',
 '?sid=NV',
 '?sid=NH',
 '?sid=NJ',
 '?sid=NM',
 '?sid=NY',
 '?sid=NC',
 '?sid=ND',
 '?sid=OH',
 '?sid=OK',
 '?sid=OR',
 '?sid=PA',
 '?sid=RI',
 '?sid=SC',
 '?sid=SD',
 '?sid=TN',
 '?sid=TX',
 '?sid=UT',
 '?sid=VT',
 '?sid=VA',
 '?sid=WA',
 '?sid=WV',
 '?sid=WI',
 '?sid=WY']

In [15]:
## construct the state_links for a sample state and parse the html data

print(base_url + state_links[47] + '#SupplyDistribution') # Test getting the data for a state for testing purpose
r = requests.get(base_url + state_links[47] + '#SupplyDistribution')
soup = BeautifulSoup(r.text, 'html.parser')

https://www.eia.gov/state/data.php?sid=WV#SupplyDistribution


In [16]:
## Get the state name, the state name is under the 'h1' tag
state = soup.find_all('h1')[1].text.replace('\xa0', '')

print(state)

West Virginia


In [17]:
## The data we need is in tr tag
rows = soup.find_all('tr')
print(len(rows))

119


In [18]:
## we need the rows that contains 'Total Net Electricity Generation'
for row in rows:
    if 'Total Net Electricity Generation' in str(row):
        target_rows = row
        electricity_generation = float(target_rows.find_all('td')[1].text.split()[0].replace(',', ''))
print(electricity_generation)

4124.0


In [19]:
## We also need data for 'Petroleum-Fired', 'Natural Gas-Fired', 'Coal-Fired', 'Nuclear', 'Renewables'

values = []  # here the target values will be added

items =['Total Net Electricity Generation', 'Petroleum-Fired', 'Natural Gas-Fired', 'Coal-Fired', 'Nuclear', 'Renewables']
for item in items:
    for row in rows:
        if item in str(row):
            target_rows = row
            try:   
                value = float(target_rows.find_all('td')[1].text.split()[0].replace(',', ''))
                values.append(value)
            except:   #some items has got value  '*', in that case append 0
                values.append(0)
print(values)



[4124.0, 0.5, 2.4, 88.7, 0.0, 8.3]


In [20]:
## make a data dictionary for the test state
data = {}
data[state] = values
print(data)

{'West Virginia': [4124.0, 0.5, 2.4, 88.7, 0.0, 8.3]}


In [21]:
## new iterate for all the 50 states to get data
data = {}  #dictionary which will contain the state name and target dtaa

for link in state_links:
    r = requests.get(base_url + link + '#SupplyDistribution')  #construct complete link
    soup = BeautifulSoup(r.text, 'html.parser')
    
    ## Get the state name, the state name is under the 'h1' tag
    state = soup.find_all('h1')[1].text.replace('\xa0', '')
    print(f'Accessing data for {state}..')
    
    ## The data we need is in tr tag
    rows = soup.find_all('tr')

    ## We also need data for 'Petroleum-Fired', 'Natural Gas-Fired', 'Coal-Fired', 'Nuclear', 'Renewables'
    values = []  # here the target values will be added
    items =['Total Net Electricity Generation', 'Petroleum-Fired', 'Natural Gas-Fired', 'Coal-Fired', 'Nuclear', 'Renewables']
    for item in items:
        for row in rows:
            if item in str(row):
                target_rows = row
                try:   
                    value = float(target_rows.find_all('td')[1].text.split()[0].replace(',', ''))
                    values.append(value)
                except:   #some items has got value  '*', in that case append 0
                    values.append(0)
    
    data[state] = values

print('\nObtained data:')
print(data)

Accessing data for Alabama..
Accessing data for Alaska..
Accessing data for Arizona..
Accessing data for Arkansas..
Accessing data for California..
Accessing data for Colorado..
Accessing data for Connecticut..
Accessing data for Delaware..
Accessing data for Florida..
Accessing data for Georgia..
Accessing data for Hawaii..
Accessing data for Idaho..
Accessing data for Illinois..
Accessing data for Indiana..
Accessing data for Iowa..
Accessing data for Kansas..
Accessing data for Kentucky..
Accessing data for Louisiana..
Accessing data for Maine..
Accessing data for Maryland..
Accessing data for Massachusetts..
Accessing data for Michigan..
Accessing data for Minnesota..
Accessing data for Mississippi..
Accessing data for Missouri..
Accessing data for Montana..
Accessing data for Nebraska..
Accessing data for Nevada..
Accessing data for New Hampshire..
Accessing data for New Jersey..
Accessing data for New Mexico..
Accessing data for New York..
Accessing data for North Carolina..
Acce

In [18]:
print(f'Successfully aobtained data from {len(data)} states')

Successfully aobtained data from 50 states


In [14]:
##save the data in .csv file

import csv

with open('Electricity production mix.csv','w') as f:
    w = csv.writer(f)
    w.writerows(data.items())
#column names    
#'Total Net Electricity Generation(thousand MWh)', 'Petroleum-Fired(%)', 'Natural Gas-Fired(%)',
#'Coal-Fired(%)', 'Nuclear(%)', 'Renewables(%)'